# LIBRARIES

In [1]:
import pandas as pd
import numpy  as np
import time
import matplotlib.pyplot as plt

from pkg.constants    import *
from pkg.evaluation   import Evaluation
from pkg.utils        import *
from pkg.algorithims  import *
from pkg.neighborhood import *



from pkg.tabu_list import TabuList

from itertools import permutations
from random    import shuffle



# SEED PRODUCTION

In [2]:
# seeds = np.random.randint(0,99999,SEEDS)
# print(seeds)

# Semillas adri
seeds = np.array([12345, 54321, 12221, 22211, 22212])
print(seeds)

[12345 54321 12221 22211 22212]


# EVALUATION OBJECT

In [3]:
evaluation = Evaluation()

# IMPLEMENATION OF ALGORITHIMS

## DATA DUMP

In [4]:
deltas_df   = pd.read_csv(DELTAS_5M_PATH) 
deltas_matrix   = deltas_df.to_numpy()

## GREEDY SEARCH

In [5]:
initial_state = deltas_matrix[0,:]
start_time = time.time()
greedy_sol, greedy_sol_cost = greedy(initial_state, evaluation)
finish_time = time.time()
print(finish_time - start_time)

0.07283949851989746


In [6]:
print(greedy_sol)
print(greedy_sol_cost)

[ 7.  9. 18.  8. 11. 18. 11. 12.  8. 13. 13. 24. 11. 18. 20. 19.]
574.5513854177129


## RANDOM SEARCH

In [9]:
for seed in seeds:
    np.random.seed(seed)
    rs_sol, rs_sol_cost = random_search(evaluation = evaluation)

    print(rs_sol, round(rs_sol_cost,4))


[14. 14. 14. 24. 11.  5.  8. 16. 14. 19. 19. 19. 14.  8.  8. 14.] 437.9733
[ 9. 15. 15. 19. 15. 19. 11. 15. 13. 17. 19. 11.  9.  6. 13. 15.] 451.3225
[14. 12. 14. 18. 12. 16. 18. 12. 18.  4. 18. 18. 12. 10. 14. 16.] 414.6951
[18.  6. 12. 12. 12. 24. 15. 12.  6. 18. 15. 21.  6. 21.  6. 18.] 437.2946
[21. 10. 10. 13. 21.  5.  8. 13.  8. 16. 16. 21. 10. 10. 21. 18.] 437.6238


## LOCAL SEARCH
first best neighbor

In [10]:
granularity = 1

for i in range(SEEDS):

    np.random.seed(seeds[i])

    ls_sol, ls_sol_cost, calls = local_search(
        granularity = granularity,
        evaluation  = evaluation
    )
    
    print(ls_sol, round(ls_sol_cost,4), calls)

  

[18. 12. 12. 14. 20. 14. 14. 12. 10.  8. 12. 25. 15.  2.  6. 25.] 411.7068 919
[12. 12. 20. 14. 19. 14. 13.  8. 11. 14. 16. 24.  2.  7. 22. 12.] 372.4192 2056
[16. 12. 19. 14. 20. 14. 10. 12. 12. 12. 12. 24.  2. 13. 17. 12.] 361.1852 1086
[16. 10. 22. 18. 20. 14. 14. 12.  9.  8. 12. 18.  2. 22.  7. 12.] 411.2291 874
[18. 13. 12. 15. 20. 14. 21. 12. 10.  8. 24. 24.  8.  2.  6. 12.] 417.9571 733


## SIMULATED ANNEALING

In [11]:
# parametros a ajustar:
slots = 1

# k_max: maximo numero iteraciones: en el entorno de 80
# condicion de parada
k_max = 80

# L(T): condicion enfriamiento. Numero fijo para cada iteracion
# Se enfria la temperatura al final de una iteracion
# La iteracion finaliza al visitar todos los vecinos
L = 20


# Esquema de enfriamiento: Cauchy
# tk = t0/(1+k)

# mu: tanto por uno [0,1], indica la probabilidad de aceptar una solucion peor que la inicial
mu = 0.2

# fi: probabilidad de aceptar una solucion un mu por 1 peor que la inicial
fi = 0.2

# Temperatura inicial
t0 = mu * greedy_sol_cost / (- np.log(fi))

# Temperatura final
tf = t0 / (1 + k_max)



In [13]:
for i in range(SEEDS):
    np.random.seed(seeds[i])
    sa_solution, sa_cost = simulated_an(t0,L,tf,slots,evaluation)
    print(sa_solution, round(sa_cost, 4))



[15. 12. 20. 20. 16. 15. 15.  7. 10. 14. 15. 24.  2. 11. 11. 12.] 385.4951
[10. 11. 20. 17. 20. 14. 15. 11. 10. 12. 15. 27.  2. 14. 11. 11.] 384.5774
[18. 12. 20. 17. 20. 14.  8. 12. 12. 11. 12. 23.  2. 20.  8. 12.] 359.8482
[15. 10. 19. 25. 20. 14. 14. 12. 13. 14. 12. 19.  2.  7.  8. 12.] 412.6862
[11. 13. 22. 19. 18. 14. 14.  9. 10. 14. 15. 24.  2. 16.  4. 14.] 385.1344


##  TABU SEARCH

In [16]:
# externo
reboots = 4
total_iterations = 80

total_neighbors = 40
tenure = 4

slots = 3 # esto por que????


In [17]:
for seed in seeds:
    np.random.seed(seed)
    ts_sol, ts_sol_cost = tabu_search(
        tenure           = tenure,
        reboots          = reboots,
        total_iterations = total_iterations,
        slots            = slots,
        total_neighbors  = total_neighbors,
        evaluation       = evaluation
    )

    print(ts_sol, round(ts_sol_cost, 4))

[17. 10. 19. 16. 18. 11. 12. 11. 11. 10. 14. 24.  5. 22.  7. 13.] 384.6226
[14. 13. 19. 17. 18. 14. 14.  7. 11. 20. 17. 22.  3.  5. 12. 14.] 384.7484
[16. 13. 17. 16. 20. 13. 13. 11. 14. 11. 13. 19.  4.  8. 22. 11.] 374.9986
[19. 11. 13. 16. 19. 16.  7. 11. 14. 15. 14. 28.  2. 13. 11. 12.] 377.1583
[14. 14. 17. 20. 18. 14. 13.  6. 10. 17. 19. 24.  2.  8. 11. 14.] 372.3626


In [17]:
def generate_greedy_solution(matrix):

    solution = np.empty(matrix.shape[1])

    # calculamos la inversa
    inversa = 1/matrix

    
    for i in range(matrix.shape[1]):

        sumatorio = inversa[:,i].sum()

        # normalizamos la columna
        inversa[:,i] = inversa[:,i]/sumatorio

        num = np.random.random()
        sum = 0
        control = 0

        for row in range(matrix.shape[0]):
            sum += inversa[row,i]
            if num < sum:
                solution[i] = row
                control = row # indica cuantos slots se han metido
                break
    
        # reajuste de matriz
        if row != 0:
            inversa = inversa[:-row, :]
        if row is matrix.shape[0]-1:
            pass
    
    return solution
        
        

    


In [2]:
valores_base = np.array(range(1,50,3))
matriz_frecuencias = np.ones( (len( valores_base ), TOTAL_STATIONS) )
capacidad = np.array([8,8,8,8,8,8,8,8,8,8,8,8,8,8,8,8])

print(valores_base)
print(update_frequency_matrix(matriz_frecuencias, capacidad, valores_base))


[ 1  4  7 10 13 16 19 22 25 28 31 34 37 40 43 46 49]
[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]


# ALGORITHMS ADJUSTMENT